В домашнем задании нужно решить задачу классификации наличия болезни сердца у пациентов. Целевая переменная – наличие болезни сердца (HeartDisease), принимает значения 0 или 1 в зависимости от отсутствия или наличия болезни соответственно.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, StackingClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

1. Получите данные и загрузите их в рабочую среду. (Jupyter Notebook или другую)

In [ ]:
data = pd.read_csv('heart.csv')
data.head()

In [ ]:
data.info()

In [ ]:
data['RestingBP'].sort_values().unique()

In [ ]:
data['Cholesterol'].sort_values().unique()

2. Подготовьте датасет к обучению моделей.
* Категориальные переменные переведите в цифровые значения. Можно использовать pd.get_dummies, preprocessing.LabelEncoder. Старайтесь не использовать для этой задачи циклы.
* Постройте 1-2 графика на выбор. Визуализация должна быть основана на исследуемых данных и быть полезной (из графика можно сделать вывод об особенностях датасета/класса/признака).

In [ ]:
data.columns

In [ ]:
# Для начала оценим частоту встречаемости болезни сердца у пациентов в зависимости от их возраста

data_disease = data[data['HeartDisease'] == 1]
sns.histplot(data_disease['Age'], kde=True)

plt.xlabel('Age')
plt.ylabel('Частота')
plt.title('KDE-распределение признака Age')
plt.show()

mode_age_disease = data_disease['Age'].mode()[0]
print(f'Возрастная группа риска пациентов с HeartDisease=1: {mode_age_disease}')

In [ ]:
# Затем количественно определим кто больше подвержен заболеванию - женщины или мужчины 

plt.figure(figsize=(8, 6))
sns.countplot(data=data, x='Sex', hue='HeartDisease')
plt.title('Количественная зависимость пола и сердечных заболеваний')
plt.xlabel('Пол')
plt.ylabel('Количество')
plt.legend(title='Heart Disease Class')
plt.show()

In [ ]:
# Оценим тип боли в груди при наличии/отсутствии заболевания

plt.figure(figsize=(8, 6))
sns.countplot(data=data, x='ChestPainType', hue='HeartDisease')
plt.title('Гистограмма типов боли в груди в зависимости от сердечного заболевания')
plt.xlabel('Тип боли в груди')
plt.ylabel('Количество')
plt.legend(title='Heart Disease Class')
description = 'TA: Typical Angina, ATA: Atypical Angina, NAP: Non-Anginal Pain, ASY: Asymptomatic'
plt.figtext(0.5, 0.01, description, ha='center', fontsize=10)
plt.show()

In [ ]:
sns.kdeplot(data[data['HeartDisease'] == 0]['Cholesterol'], fill=True, color="blue",label='HeartDisease=0')
sns.kdeplot(data[data['HeartDisease'] == 1]['Cholesterol'], fill=True, color="red",label='HeartDisease=1')
plt.legend()
plt.xlabel('Холестерин')
plt.ylabel('Частота')
plt.title('KDE-распределение уровня холестерина в крови')
plt.show()

In [ ]:
sns.kdeplot(data[data['HeartDisease'] == 0]['MaxHR'], fill=True, color="blue",label='HeartDisease=0')
sns.kdeplot(data[data['HeartDisease'] == 1]['MaxHR'], fill=True, color="red",label='HeartDisease=1')
plt.legend()
plt.xlabel('ЧСС')
plt.ylabel('Частота')
plt.title('KDE-распределение частоты сердцебиения')
plt.show()

In [ ]:
corr_matrix = data.corr()
plt.figure(figsize=(8,6))
sns.heatmap(corr_matrix, annot=True, fmt='.2f')

plt.title('Корреляционная матрица признаков')

In [ ]:
data_encoded = pd.get_dummies(data, columns=['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'])
data_encoded.head()

3. Разделите выборку на обучающее и тестовое подмножество. 80% данных оставить на обучающее множество, 20% на тестовое.

In [ ]:
data_encoded.columns

In [ ]:
X = data_encoded.drop('HeartDisease',axis=1)
y = data_encoded['HeartDisease']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

4. Обучите дерево решений на обучающем множестве. Используйте следующие модели:
* tree.DecisionTreeClassifier
* ensemble.RandomForestClassifier
5. Для тестового множества сделайте предсказание целевой переменной. Выведите метрики для каждой построенной модели с помощью metrics.classification_report.

In [ ]:
tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)

In [ ]:
y_train_pred = tree.predict(X_train)
y_test_pred = tree.predict(X_test)

In [ ]:
# Наблюдаем переобучение

report_train = classification_report(y_train, y_train_pred)
print(report_train)

In [ ]:
report = classification_report(y_test, y_test_pred)
print(report)

In [ ]:
# Зададим параметры для поиска про сетке с помощью GridSearchCV

param_grid = {
    'criterion': ['gini', 'entropy'],      
    'max_depth': [3, 5, 10, None],        
    'min_samples_split': [2, 5, 10],     
    'min_samples_leaf': [1, 2, 4],
}

grid_search = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_grid,
    cv=5,                
    scoring='accuracy',  
    n_jobs=-1,           
    verbose=1     
)

grid_search.fit(X_train, y_train)

print(f'Лучшие параметры: {grid_search.best_params_}')
print(f'Accuracy: {grid_search.best_score_:.2f}')

best_model = grid_search.best_estimator_
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

In [ ]:
report_train = classification_report(y_train, y_train_pred)
print(report_train)

In [ ]:
report = classification_report(y_test, y_test_pred)
print(report)

In [ ]:
randomforest = RandomForestClassifier(random_state=42)
randomforest.fit(X_train, y_train)

In [ ]:
y_train_pred = randomforest.predict(X_train)
y_test_pred = randomforest.predict(X_test)

In [ ]:
report_train = classification_report(y_train, y_train_pred)
print(report_train)

In [ ]:
report = classification_report(y_test, y_test_pred)
print(report)

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False],
}

grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

print(f'Лучшие параметры: {grid_search.best_params_}')
print(f'Accuracy: {grid_search.best_score_:.2f}')

best_model = grid_search.best_estimator_
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

In [ ]:
report_train = classification_report(y_train, y_train_pred)
print(report_train)

In [ ]:
report = classification_report(y_test, y_test_pred)
print(report)

6. Выведите важность признаков, полученную после обучения модели из п. 4b в виде столбчатой диаграммы. Отсортируйте важность по убыванию.

In [ ]:
importances = pd.Series(randomforest.feature_importances_)

In [ ]:
importances.index = pd.Series(X_train.columns)

In [ ]:
importances = importances.sort_values(ascending=False)
importances

7. Обучите бэггинг над моделью из п. 4a. Используйте ensemble.BaggingClassifier. Повторите п. 5

In [ ]:
bagging = BaggingClassifier(DecisionTreeClassifier(),
                            n_estimators=50,
                            max_samples=0.8,
                            max_features=1,
                            bootstrap=True,
                            random_state=42)

bagging.fit(X_train, y_train)
y_test_pred = bagging.predict(X_test)

report = classification_report(y_test, y_test_pred)
print(report)

8. Обучите стекинг трех моделей: из п. 4a, п. 4b и svm.LinearSVC. Используйте ensemble.StackingClassifier. Повторите п. 5

In [ ]:
stacking = StackingClassifier(
    [
        ('DecisionTree', DecisionTreeClassifier(random_state=42)),
        ('RandomForest', RandomForestClassifier(n_estimators=100, random_state=42)),
        ('SVM', make_pipeline(StandardScaler(), LinearSVC(max_iter=10000, random_state=42)))
    ], LogisticRegression())

stacking.fit(X_train, y_train)
y_test_pred = stacking.predict(X_test)

report = classification_report(y_test, y_test_pred)
print(report)

9. Сформулируйте выводы по проделанной работе.
* Сравните метрики построенных моделей.
* Напишите свое мнение, какая модель наилучшая и почему.

### Лучший результат показал алгоритм случайного леса RandomForestClassifier с подбором наилучших гиперпараметров модели по сетке GridSearchCV.